### Singular Value Decomposition (SVD) and Randomized SVD

The SVD algorithm factorizes a matrix into one matrix with orthogonal columns and one with orthogonal rows (along with a diagonal matrix, which contains the relative importance of each factor).

SVD is excellent, but it can be very slow. To save time, we can use another method called Randomized SVD which performs SVD on a smaller matrix that has approximately the same range as our original matrix and gets an acceptable result much faster.

Below are some examples of applying SVD and randomized SVD on topic modeling.

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline
np.set_printoptions(suppress=True)

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, remove=remove)

In [3]:
# Create matrix from data

vectorizer = CountVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(newsgroups_train.data).todense() # (documents, vocab)
vectors.shape #, vectors.nnz / vectors.shape[0], row_means.shape

(2034, 26576)

In [4]:
vocab = np.array(vectorizer.get_feature_names())

In [5]:
num_top_words=8

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = [top_words(t) for t in a]
    return [' '.join(t) for t in topic_words]

### full SVD

In [6]:
from scipy import linalg

%time U, s, Vh = linalg.svd(vectors, full_matrices=False)

CPU times: user 58.7 s, sys: 1.62 s, total: 1min
Wall time: 16.4 s


In [7]:
show_topics(Vh[:10])

['ditto critus propagandist surname galacticentric kindergarten surreal imaginative',
 'jpeg gif file color quality image jfif format',
 'graphics edu pub mail 128 3d ray ftp',
 'jesus god matthew people atheists atheism does graphics',
 'image data processing analysis software available tools display',
 'god atheists atheism religious believe religion argument true',
 'space nasa lunar mars probe moon missions probes',
 'image probe surface lunar mars probes moon orbit',
 'argument fallacy conclusion example true ad argumentum premises',
 'space larson image theory universe physical nasa material']

### randomized SVD

##### 1. sklearn

In [8]:
from sklearn import decomposition

%time U, s, Vh = decomposition.randomized_svd(vectors, 10)

CPU times: user 14.1 s, sys: 3.16 s, total: 17.3 s
Wall time: 5.75 s


In [9]:
show_topics(Vh)

['jpeg image edu file graphics images gif data',
 'jpeg gif file color quality image jfif format',
 'space jesus launch god people satellite matthew atheists',
 'jesus god matthew people atheists atheism does graphics',
 'image data processing analysis software available tools display',
 'jesus matthew prophecy messiah psalm isaiah david said',
 'launch commercial satellite market image services satellites launches',
 'data available nasa ftp grass anonymous contact gov',
 'argument fallacy conclusion example true ad argumentum premises',
 'probe data surface moon mars probes lunar launch']

##### 2. fbpca

In [10]:
import fbpca

%time  U, s, Vh = fbpca.pca(vectors, 10)

CPU times: user 5.3 s, sys: 1.14 s, total: 6.45 s
Wall time: 2.05 s


In [11]:
show_topics(Vh)

['kent cheers bobby islamic muslim ico manhattan prize',
 'jpeg gif file color quality image jfif bit',
 'graphics edu pub mail 128 3d ray send',
 'jesus god matthew people atheists atheism does religious',
 'image data processing analysis software available tools display',
 'god atheists atheism religious believe religion argument atheist',
 'nasa space lunar mars probe moon missions available',
 'data available nasa ftp god jesus information anonymous',
 'argument fallacy conclusion example true larson argumentum ad',
 'probe surface mars moon lunar data probes launch']

##### 3. simple implement

In [12]:
def simple_randomized_svd(M, k=10):
    m, n = M.shape
    transpose = False
    if m < n:
        transpose = True
        M = M.T
        
    rand_matrix = np.random.normal(size=(M.shape[1], k))  # short side by k
    Q, _ = np.linalg.qr(M @ rand_matrix, mode='reduced')  # long side by k
    smaller_matrix = Q.T @ M                              # k by short side
    U_hat, s, V = np.linalg.svd(smaller_matrix, full_matrices=False)
    U = Q @ U_hat
    
    if transpose:
        return V.T, s.T, U.T
    else:
        return U, s, V

In [13]:
%time  U, s, Vh = simple_randomized_svd(vectors, 10)

CPU times: user 1.63 s, sys: 334 ms, total: 1.97 s
Wall time: 622 ms


In [14]:
show_topics(Vh.tolist())

['image jpeg edu data file images graphics gif',
 'edu data nasa launch space ftp graphics pub',
 'graphics edu pub mail 3d ftp fallacy ray',
 'jesus people like edu just good god say',
 'image data processing software center display analysis line',
 'jesus data ra space available gay man matthew',
 'ra image space launch fallacy jesus sci support',
 'den tyre make p3 available p2 example people',
 'ra space edu den p2 p3 sci atheism',
 'astronaut edu mars faq military says den alt']